In [1]:
import sys
import subprocess
import importlib

# ---- CONFIGURATION ----
REQUIRED_PYTHON = (3, 9)
REQUIRED_PACKAGES = [
    "pandas",
    "openpyxl",
    "sqlalchemy",
    "psycopg[binary]",
    "jupyterlab",
]

# ---- PYTHON VERSION CHECK ----
if sys.version_info < REQUIRED_PYTHON:
    raise SystemExit(
        f"❌ Python {REQUIRED_PYTHON[0]}.{REQUIRED_PYTHON[1]}+ required, "
        f"found {sys.version_info.major}.{sys.version_info.minor}"
    )
else:
    print(f"✅ Python version OK: {sys.version_info.major}.{sys.version_info.minor}")

# ---- PACKAGE CHECK / AUTO-INSTALL ----
def install(pkg):
    """Install a package via pip in the current environment."""
    print(f"⬇️  Installing missing package: {pkg}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

missing = []
for pkg in REQUIRED_PACKAGES:
    # psycopg[binary] isn't a real import name, so handle that separately
    import_name = pkg.split("[")[0]
    try:
        importlib.import_module(import_name)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f"\n⚠️ Missing packages detected: {missing}")
    for pkg in missing:
        install(pkg)
else:
    print("✅ All required packages are already installed.")

# ---- SHOW INSTALLED VERSIONS ----
print("\n📦 Installed versions:")
for pkg in REQUIRED_PACKAGES:
    pkg_base = pkg.split("[")[0]
    try:
        mod = importlib.import_module(pkg_base)
        version = getattr(mod, "__version__", "unknown")
        print(f"  - {pkg_base} == {version}")
    except ImportError:
        print(f"  - {pkg_base} not installed")


✅ Python version OK: 3.10


✅ All required packages are already installed.

📦 Installed versions:
  - pandas == 2.3.3
  - openpyxl == 3.1.5
  - sqlalchemy == 2.0.44
  - psycopg == 3.2.12
  - jupyterlab == 4.4.10


In [109]:
!pip freeze > requirements.txt
print("✅ requirements.txt updated")

✅ requirements.txt updated


In [117]:
# --- CONFIG ---
from pathlib import Path

# PostgreSQL
PG_USER = "postgres"
PG_PASS = "greenage"
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "moin_weather"   # make sure DB is UTF8
PG_SCHEMA = "lgs2"
TABLE_NAME = "varieties_stages"

# Excel source
EXCEL_PATH = Path(r"./Crop Growth Models - Wheat (1).xlsx")
SHEET_NAME = "Crop_Phenology"
USECOLS    = "B:P"     # adjust if needed
HEADER_ROW = 1          # Excel row 4 -> pandas header=3 (0-indexed)

# --- ENGINE ---
from sqlalchemy import create_engine, text

conn_str = f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
# client_encoding can be SET after connect; psycopg3 also accepts options, but SET works reliably
engine = create_engine(conn_str, pool_pre_ping=True)

with engine.connect() as conn:
    server_enc = conn.execute(text("SHOW SERVER_ENCODING;")).scalar_one()
    conn.execute(text("SET client_encoding TO 'UTF8';"))
    client_enc = conn.execute(text("SHOW CLIENT_ENCODING;")).scalar_one()
    print("SERVER_ENCODING:", server_enc)
    print("CLIENT_ENCODING:", client_enc)


SERVER_ENCODING: UTF8
CLIENT_ENCODING: UTF8


In [118]:
import pandas as pd

# Read as strings first to keep full control over conversion
df_raw = pd.read_excel(
    EXCEL_PATH,
    sheet_name=SHEET_NAME,
    header=HEADER_ROW,
    usecols=USECOLS,
    dtype=str,
    engine="openpyxl",
)

# strip whitespace in *all* string cells
df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# drop fully empty rows
df_raw = df_raw.dropna(how="all")

print("Shape after load & trim:", df_raw.shape)
print("Raw columns:")
print(df_raw.columns.tolist())

display(df_raw.head())


C:\Users\ali_raza\AppData\Local\Temp\ipykernel_15576\4073561252.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_raw = df_raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Shape after load & trim: (263, 15)
Raw columns:
['Crop_Name', 'Variety_Name', 'Principal_Stage', 'Main_Stage', 'Sub_Stage', 'Start_GDD', 'End_GDD', 'Start_Day', 'End_Day', 'Daily_N_Kg/ha', 'Daily_P_Kg/ha', 'Daily_K_Kg/ha', 'Crop_Coefficient', 'K_Ext (PAR)', 'DM:FM']


,Crop_Name,Variety_Name,Principal_Stage,Main_Stage,Sub_Stage,Start_GDD,End_GDD,Start_Day,End_Day,Daily_N_Kg/ha,Daily_P_Kg/ha,Daily_K_Kg/ha,Crop_Coefficient,K_Ext (PAR),DM:FM
0,Wheat,bars-2009,Germination,Germination,00: Dry seed,0,10,0,0,0.05,0.01,0.05,0.28,0.4,0
1,Wheat,bars-2009,Germination,Germination,01: Beginning of seed imbibition,11,40,1,1,NaN,NaN,NaN,NaN,NaN,0
2,Wheat,bars-2009,Germination,Germination,03: Seed imbibition complete,41,70,2,2,NaN,NaN,NaN,NaN,NaN,0
3,Wheat,bars-2009,Germination,Germination,05: Radicle Emerged from seed,71,95,3,3,NaN,NaN,NaN,NaN,NaN,0
4,Wheat,bars-2009,Germination,Germination,07: Hypocotyl with cotyledons breaking through...,96,110,4,4,NaN,NaN,NaN,NaN,NaN,0


In [119]:
import re

def to_snake(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[^\w\s]+", " ", s)  # punctuation -> spaces
    s = re.sub(r"\s+", "_", s)       # whitespace -> underscore
    return s.lower()

snake_map = {c: to_snake(c) for c in df_raw.columns}

# Apply initial snake-case
df_snake = df_raw.rename(columns=snake_map)

# --- Your custom overrides (keys are snake-case) ---
# e.g. raw "Crop Name" -> snake "crop_name" -> override to "crop_fk"
custom_overrides = {
    # "crop_name": "crop_fk",
    "variety_name": "variety_fk",
    "k_ext_par_": "k_ext_par",
    # "hi": "harvest_index",
    # "opt__temp": "opt_temp"
    }

# Apply overrides
rename_map = {**{v: v for v in df_snake.columns}, **custom_overrides}
df_renamed = df_snake.rename(columns=rename_map)

print("Column rename preview:")
display(pd.DataFrame({
    "original": list(df_raw.columns),
    "snake": [snake_map[c] for c in df_raw.columns],
    "final": [rename_map.get(snake_map[c], snake_map[c]) for c in df_raw.columns],
}))

print("Renamed columns:")
print(df_renamed.columns.tolist())
display(df_renamed.head())


Column rename preview:


,original,snake,final
0,Crop_Name,crop_name,crop_name
1,Variety_Name,variety_name,variety_fk
2,Principal_Stage,principal_stage,principal_stage
3,Main_Stage,main_stage,main_stage
4,Sub_Stage,sub_stage,sub_stage
5,Start_GDD,start_gdd,start_gdd
6,End_GDD,end_gdd,end_gdd
7,Start_Day,start_day,start_day
8,End_Day,end_day,end_day
9,Daily_N_Kg/ha,daily_n_kg_ha,daily_n_kg_ha


Renamed columns:
['crop_name', 'variety_fk', 'principal_stage', 'main_stage', 'sub_stage', 'start_gdd', 'end_gdd', 'start_day', 'end_day', 'daily_n_kg_ha', 'daily_p_kg_ha', 'daily_k_kg_ha', 'crop_coefficient', 'k_ext_par', 'dm_fm']


,crop_name,variety_fk,principal_stage,main_stage,sub_stage,start_gdd,end_gdd,start_day,end_day,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,dm_fm
0,Wheat,bars-2009,Germination,Germination,00: Dry seed,0,10,0,0,0.05,0.01,0.05,0.28,0.4,0
1,Wheat,bars-2009,Germination,Germination,01: Beginning of seed imbibition,11,40,1,1,NaN,NaN,NaN,NaN,NaN,0
2,Wheat,bars-2009,Germination,Germination,03: Seed imbibition complete,41,70,2,2,NaN,NaN,NaN,NaN,NaN,0
3,Wheat,bars-2009,Germination,Germination,05: Radicle Emerged from seed,71,95,3,3,NaN,NaN,NaN,NaN,NaN,0
4,Wheat,bars-2009,Germination,Germination,07: Hypocotyl with cotyledons breaking through...,96,110,4,4,NaN,NaN,NaN,NaN,NaN,0


In [120]:
# Put your exact desired order here (subset or superset is OK)
REQUIRED_ORDER = [
    # ---- identifiers / descriptors ----
    "uuid",
    "main_stage",
    "principal_stage",
    "bbch_scale",
    "daily_n_kg_ha",
    "daily_p_kg_ha",
    "daily_k_kg_ha",
    "crop_coefficient",
    "k_ext_par",
    "variety_fk",
    # ---- batch timestamps ----
    "created_at",
    "updated_at",
]

# Ensure presence; create missing with NA
for col in REQUIRED_ORDER:
    if col not in df_renamed.columns:
        df_renamed[col] = pd.NA

# Put required columns first, keep any extras at the end (in their current order)
ordered = [c for c in REQUIRED_ORDER if c in df_renamed.columns]
extras  = [c for c in df_renamed.columns if c not in ordered]
df_ordered = df_renamed[ordered + extras].copy()

print("Final order (first 11 shown):", (ordered + extras)[:11], "...")
display(df_ordered.head())


Final order (first 11 shown): ['uuid', 'main_stage', 'principal_stage', 'bbch_scale', 'daily_n_kg_ha', 'daily_p_kg_ha', 'daily_k_kg_ha', 'crop_coefficient', 'k_ext_par', 'variety_fk', 'created_at'] ...


,uuid,main_stage,principal_stage,bbch_scale,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,variety_fk,created_at,updated_at,crop_name,sub_stage,start_gdd,end_gdd,start_day,end_day,dm_fm
0,<NA>,Germination,Germination,<NA>,0.05,0.01,0.05,0.28,0.4,bars-2009,<NA>,<NA>,Wheat,00: Dry seed,0,10,0,0,0
1,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,bars-2009,<NA>,<NA>,Wheat,01: Beginning of seed imbibition,11,40,1,1,0
2,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,bars-2009,<NA>,<NA>,Wheat,03: Seed imbibition complete,41,70,2,2,0
3,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,bars-2009,<NA>,<NA>,Wheat,05: Radicle Emerged from seed,71,95,3,3,0
4,<NA>,Germination,Germination,<NA>,NaN,NaN,NaN,NaN,NaN,bars-2009,<NA>,<NA>,Wheat,07: Hypocotyl with cotyledons breaking through...,96,110,4,4,0


In [121]:
import pandas as pd
from decimal import Decimal, ROUND_HALF_UP

# ---- classify columns ----
INT_COLS = [
    "bbch_scale"
]

NUMERIC_2DP = [
    "daily_n_kg_ha",
    "daily_p_kg_ha",
    "daily_k_kg_ha",
    "crop_coefficient",
    "k_ext_par",
]

TS_COLS = [
    "created_at",
    "updated_at"
    ]

data = df_ordered.copy()

# ---- normalize strings ----
string_cols = [c for c in data.columns if c not in INT_COLS + NUMERIC_2DP + TS_COLS]
for c in string_cols:
    data[c] = data[c].astype("string").str.strip().str.lower()

# ---- integers ----
for c in INT_COLS:
    if c in data.columns:
        data[c] = pd.to_numeric(data[c], errors="coerce").astype("Int64")

# ---- helper to fix scale with Decimal ----
def to_decimal_series(s: pd.Series, places: int):
    q = Decimal("0." + "0"*places) if places > 0 else Decimal("0")
    return (pd.to_numeric(s, errors="coerce")
              .round(places)                              # numeric rounding (optional)
              .astype("object")                           # switch out of float dtype
              .map(lambda x: None if pd.isna(x) else Decimal(str(x)).quantize(q, rounding=ROUND_HALF_UP)))

# ---- 2dp & 3dp numerics as Decimal ----
for c in NUMERIC_2DP:
    if c in data.columns:
        data[c] = to_decimal_series(data[c], 2)

# ---- timestamps ----
current_ts = pd.Timestamp.now(tz="UTC")
data["created_at"] = current_ts
data["updated_at"] = current_ts

# ---- ensure NULLs ----
data = data.where(pd.notna(data), None)

print("✅ Cleaning complete. dtypes summary:")
display(data.dtypes)
display(data.head())


✅ Cleaning complete. dtypes summary:


uuid                     string[python]
main_stage               string[python]
principal_stage          string[python]
bbch_scale                        Int64
daily_n_kg_ha                    object
daily_p_kg_ha                    object
daily_k_kg_ha                    object
crop_coefficient                 object
k_ext_par                        object
variety_fk               string[python]
created_at          datetime64[us, UTC]
updated_at          datetime64[us, UTC]
crop_name                string[python]
sub_stage                string[python]
start_gdd                string[python]
end_gdd                  string[python]
start_day                string[python]
end_day                  string[python]
dm_fm                    string[python]
dtype: object

,uuid,main_stage,principal_stage,bbch_scale,daily_n_kg_ha,daily_p_kg_ha,daily_k_kg_ha,crop_coefficient,k_ext_par,variety_fk,created_at,updated_at,crop_name,sub_stage,start_gdd,end_gdd,start_day,end_day,dm_fm
0,<NA>,germination,germination,<NA>,0.05,0.01,0.05,0.28,0.40,bars-2009,2025-10-29 11:16:28.918000+00:00,2025-10-29 11:16:28.918000+00:00,wheat,00: dry seed,0,10,0,0,0
1,<NA>,germination,germination,<NA>,None,None,None,None,None,bars-2009,2025-10-29 11:16:28.918000+00:00,2025-10-29 11:16:28.918000+00:00,wheat,01: beginning of seed imbibition,11,40,1,1,0
2,<NA>,germination,germination,<NA>,None,None,None,None,None,bars-2009,2025-10-29 11:16:28.918000+00:00,2025-10-29 11:16:28.918000+00:00,wheat,03: seed imbibition complete,41,70,2,2,0
3,<NA>,germination,germination,<NA>,None,None,None,None,None,bars-2009,2025-10-29 11:16:28.918000+00:00,2025-10-29 11:16:28.918000+00:00,wheat,05: radicle emerged from seed,71,95,3,3,0
4,<NA>,germination,germination,<NA>,None,None,None,None,None,bars-2009,2025-10-29 11:16:28.918000+00:00,2025-10-29 11:16:28.918000+00:00,wheat,07: hypocotyl with cotyledons breaking through...,96,110,4,4,0


In [122]:
from sqlalchemy import MetaData, Table, Column, text, ForeignKey
from sqlalchemy import String, Integer, DateTime, Numeric
from sqlalchemy.dialects.postgresql import UUID

metadata = MetaData(schema=PG_SCHEMA)

table = Table(
    TABLE_NAME, metadata,
    # identifiers
    Column("uuid",                UUID(as_uuid=True), primary_key=True),
    Column("main_stage",          String),          # short labels -> String
    Column("principal_stage",     String),
    Column("bbch_scale",          Integer),
    Column("daily_n_kg_ha",       Numeric(10, 2)),
    Column("daily_p_kg_ha",       Numeric(10, 2)),
    Column("daily_k_kg_ha",       Numeric(10, 2)),
    Column("crop_coefficient",    Numeric(6, 2)),   # kc
    Column("k_ext_par",           Numeric(6, 2)),
    Column("variety_fk",          String, ForeignKey(f"{PG_SCHEMA}.crop_varieties.variety_eng", onupdate="CASCADE", ondelete="RESTRICT", 
                                                     name="fk_varieties_stages__variety_fk__crop_varieties_variety_eng",), nullable=False),
    Column("created_at",          DateTime(timezone=True)),
    Column("updated_at",          DateTime(timezone=True)),
    schema=PG_SCHEMA,
)

with engine.begin() as conn:
    conn.execute(text(f'CREATE SCHEMA IF NOT EXISTS "{PG_SCHEMA}";'))

    # reflect the parent table so SQLAlchemy knows about it
    crop_varieties = Table(
        "crop_varieties",
        metadata,
        schema=PG_SCHEMA,
        autoload_with=conn,      # <- reflect existing structure
    )

    metadata.create_all(conn)
    print(f"✅ Ensured table {PG_SCHEMA}.{TABLE_NAME} exists (or was created).")


✅ Ensured table lgs2.varieties_stages exists (or was created).


In [123]:
from sqlalchemy import String as SA_String, Integer as SA_Integer, DateTime as SA_DateTime, Numeric as SA_Numeric
from sqlalchemy.dialects.postgresql import UUID as SA_UUID

dtype_map = {
    "uuid":              SA_UUID(as_uuid=True),
    "main_stage":        SA_String(),
    "principal_stage":   SA_String(),
    "bbch_scale":        SA_Integer(),
    "daily_n_kg_ha":     SA_Numeric(10, 2),
    "daily_p_kg_ha":     SA_Numeric(10, 2),
    "daily_k_kg_ha":     SA_Numeric(10, 2),
    "crop_coefficient":  SA_Numeric(6, 2),
    "k_ext_par":         SA_Numeric(6, 2),
    "variety_fk":        SA_String(),
    "created_at":        SA_DateTime(timezone=True),
    "updated_at":        SA_DateTime(timezone=True),
}


In [124]:
from uuid import UUID, uuid4
import pandas as pd

# --- choose one or more varieties to upload (case-insensitive) ---
TARGET_VARIETIES = [
    "bars-2009",
    "fakhar-e-bhakkar",
    "farid-2006",
    "lasani-2008",
    "narc-2011",
    "punjab-2011",
]

if "variety_fk" not in data.columns:
    raise KeyError("Expected column 'variety_fk' not found in DataFrame")

# ---- filter to target varieties (case-insensitive) ----
var_norm = data["variety_fk"].astype("string").str.strip().str.casefold()
targets_norm = [v.casefold() for v in TARGET_VARIETIES]
mask = var_norm.isin(targets_norm)

total_rows = len(data)
kept_rows = int(mask.sum())

if kept_rows == 0:
    raise ValueError(
        f"No rows matched variety_fk in {TARGET_VARIETIES}. Aborting upload."
    )
elif kept_rows < total_rows:
    others = (
        data.loc[~mask, "variety_fk"]
        .dropna().astype("string").str.strip().str.casefold().unique().tolist()
    )
    print(
        f"ℹ️ Filtering to {TARGET_VARIETIES}. "
        f"Kept {kept_rows}/{total_rows} rows. Excluded varieties present: {others}"
    )

data = data.loc[mask].copy()

# (optional) assert all remaining are in target set
assert set(
    data["variety_fk"].astype("string").str.strip().str.casefold().unique()
) <= set(targets_norm), "Found non-target variety rows after filtering."

# ---- UUID: ensure present and valid for every row ----
def _coerce_uuid(x):
    if x is None or x is pd.NA or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and x.strip() == ""):
        return uuid4()
    try:
        return x if isinstance(x, UUID) else UUID(str(x))
    except Exception:
        return uuid4()

if "uuid" not in data.columns:
    data["uuid"] = [uuid4() for _ in range(len(data))]
else:
    data["uuid"] = data["uuid"].map(_coerce_uuid)

if data["uuid"].isna().any():
    raise RuntimeError("UUID still has nulls after coercion; aborting.")

# ---- DEDUPE: keep one principal_stage per main_stage *per variety* ----
# normalize keys
data["main_stage_key"] = data["main_stage"].astype("string").str.strip().str.casefold()
data["principal_stage_key"] = data["principal_stage"].astype("string").str.strip().str.casefold()
data["variety_key"] = data["variety_fk"].astype("string").str.strip().str.casefold()

before = len(data)
data = (
    data.sort_values(["variety_key", "main_stage_key", "principal_stage_key"], kind="mergesort")
        .drop_duplicates(subset=["variety_key", "main_stage_key", "principal_stage_key"], keep="first")
        .reset_index(drop=True)
)
after = len(data)
if after < before:
    print(f"ℹ️ Dedupe: kept {after}/{before} rows after ensuring unique (variety, main_stage, principal_stage).")

# ---- BBCH SCALE: map principal stages to 0..9 *per variety* ----
BBCH_ORDER = [
    "germination",
    "leaf development (main shoot)",
    "tillering",
    "stem elongation",
    "booting",
    "inflorescence emergence (heading)",
    "flowering (anthesis)",
    "development of fruit",
    "ripening",
    "senescence",
]
_order_map = {s: i for i, s in enumerate(BBCH_ORDER)}

ps_key = data["principal_stage"].astype("string").str.strip().str.casefold()
data["bbch_scale"] = ps_key.map(_order_map).astype("Int64")

missing = data["bbch_scale"].isna().sum()
if missing:
    unmapped = sorted(ps_key[data["bbch_scale"].isna()].unique().tolist())
    print(f"⚠️ {missing} row(s) have unmapped principal_stage: {unmapped}. "
          f"Add them to BBCH_ORDER if needed.")

# ---- cleanup helper columns ----
data = data.drop(columns=["main_stage_key", "principal_stage_key", "variety_key"])

# ---- drop any columns you don’t want in DB (already handled above if needed) ----
for drop_col in [
    "crop_name", "sub_stage", "start_gdd", "end_gdd", "start_day", "end_day",
    "avg_daily_gdd", "stage_lai", "stage_rue", "stage_rootdepth_cm", "dm_fm",
]:
    if drop_col in data.columns:
        print(f"🧹 Dropping '{drop_col}' from DataFrame before upload")
        data = data.drop(columns=[drop_col])

# ---- order by variety + BBCH before upload ----
if "bbch_scale" in data.columns:
    data = (
        data.sort_values(by=["variety_fk", "bbch_scale"], kind="mergesort")
            .reset_index(drop=True)
    )
    print("✅ Data ordered by (variety_fk, bbch_scale) before upload.")
else:
    print("⚠️ bbch_scale column not found; skipping sort.")

# ---- upload ----
data.to_sql(
    name=TABLE_NAME,
    con=engine,
    schema=PG_SCHEMA,
    if_exists="append",
    index=False,
    chunksize=10_000,
    method="multi",
    dtype=dtype_map,
)

with engine.connect() as conn:
    count = conn.execute(text(f'SELECT COUNT(*) FROM "{PG_SCHEMA}"."{TABLE_NAME}";')).scalar_one()

print(
    f"✅ Upload complete (varieties={TARGET_VARIETIES}). "
    f"Row count in {PG_SCHEMA}.{TABLE_NAME}: {count}"
)

ℹ️ Filtering to ['bars-2009', 'fakhar-e-bhakkar', 'farid-2006', 'lasani-2008', 'narc-2011', 'punjab-2011']. Kept 258/263 rows. Excluded varieties present: ['variety_name']
ℹ️ Dedupe: kept 60/258 rows after ensuring unique (variety, main_stage, principal_stage).
🧹 Dropping 'crop_name' from DataFrame before upload
🧹 Dropping 'sub_stage' from DataFrame before upload
🧹 Dropping 'start_gdd' from DataFrame before upload
🧹 Dropping 'end_gdd' from DataFrame before upload
🧹 Dropping 'start_day' from DataFrame before upload
🧹 Dropping 'end_day' from DataFrame before upload
🧹 Dropping 'dm_fm' from DataFrame before upload
✅ Data ordered by (variety_fk, bbch_scale) before upload.
✅ Upload complete (varieties=['bars-2009', 'fakhar-e-bhakkar', 'farid-2006', 'lasani-2008', 'narc-2011', 'punjab-2011']). Row count in lgs2.varieties_stages: 60


In [125]:
# Pandas-side dtypes (should match what we intended)
print("Pandas dtypes:")
display(data.dtypes)

# DB-side types (optional: inspect)
qry = f"""
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_schema = :schema AND table_name = :table
ORDER BY ordinal_position;
"""
with engine.connect() as conn:
    rows = conn.execute(text(qry), {"schema": PG_SCHEMA, "table": TABLE_NAME}).mappings().all()
pd.DataFrame(rows)


Pandas dtypes:


uuid                             object
main_stage               string[python]
principal_stage          string[python]
bbch_scale                        Int64
daily_n_kg_ha                    object
daily_p_kg_ha                    object
daily_k_kg_ha                    object
crop_coefficient                 object
k_ext_par                        object
variety_fk               string[python]
created_at          datetime64[us, UTC]
updated_at          datetime64[us, UTC]
dtype: object

,column_name,data_type,is_nullable
0,uuid,uuid,NO
1,main_stage,character varying,YES
2,principal_stage,character varying,YES
3,bbch_scale,integer,YES
4,daily_n_kg_ha,numeric,YES
5,daily_p_kg_ha,numeric,YES
6,daily_k_kg_ha,numeric,YES
7,crop_coefficient,numeric,YES
8,k_ext_par,numeric,YES
9,variety_fk,character varying,NO
